In [16]:
import json
import pandas as pd
import os
import psycopg2
from shared_functions import *
import sys
from pathlib import Path
from dotenv import load_dotenv

In [7]:

table_name = 'customer_transactions'
schema_name = 'pre_prd'

In [8]:
# Get the current directory (where this script is located)
current_dir = Path(sys.path[0])
print(current_dir)


# Construct the relative path to the JSON file
file_name = 'customer_transactions[2].json'
file_path = os.path.join(current_dir, '..', 'Sources', file_name)

e:\test\Data_Engineer_Assignment\Python Script


In [9]:
with open(file_path, 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data)

In [10]:
df['transaction_id'] = df['transaction_id'].astype(str)
df['customer_id'] = df['customer_id'].astype(str)
df['product_id'] = df['product_id'].astype(str)
df['quantity'] = df['quantity'].astype(int)
df['price'] = df['price'].astype(float)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [11]:
df = df.drop_duplicates(subset='transaction_id', keep="first")

In [12]:
df

,transaction_id,customer_id,product_id,quantity,price,timestamp
0,b1182d23-5bf8-4676-9f12-a0e912011caa,C001,P003,6,150.0,2024-02-04 13:06:35
1,3b019fcb-f96d-4403-948c-93f3028d042b,C003,P001,10,100.0,2024-06-22 11:42:33
2,346d2b7e-00d3-41af-b53e-0c8624aeba79,C001,P001,5,100.0,2024-03-12 16:16:16
3,9ad7558f-eecb-425b-864f-768db080e003,C002,P002,3,200.0,2024-03-29 04:01:02
4,b51d70d8-582d-4972-ba08-9ca81cc5a8a0,C003,P002,1,200.0,2024-02-10 09:54:36


In [17]:
# Database connection details
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
USER = 'master_tk'
PASSWORD = '120344'
HOST = 'localhost'
PORT = '5432'
DATABASE = 'master_tk'

# Create the connection URL
CONNECTION_URL = f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"

# Create the SQLAlchemy engine
engine = create_engine(CONNECTION_URL)


In [18]:
insert_into_postgres(df, table_name, schema_name, engine)

Error: (psycopg2.errors.ForeignKeyViolation) insert or update on table "customer_transactions" violates foreign key constraint "customer_transaction_product_id_fkey"
DETAIL:  Key (product_id)=(P003) is not present in table "product_catalog".

[SQL: INSERT INTO pre_prd.customer_transactions (transaction_id, customer_id, product_id, quantity, price, timestamp) VALUES (%(transaction_id__0)s, %(customer_id__0)s, %(product_id__0)s, %(quantity__0)s, %(price__0)s, %(timestamp__0)s), (%(transaction_id_ ... 332 characters truncated ... ion_id__4)s, %(customer_id__4)s, %(product_id__4)s, %(quantity__4)s, %(price__4)s, %(timestamp__4)s)]
[parameters: {'quantity__0': 6, 'price__0': 150.0, 'timestamp__0': datetime.datetime(2024, 2, 4, 13, 6, 35), 'product_id__0': 'P003', 'customer_id__0': 'C001', 'transaction_id__0': 'b1182d23-5bf8-4676-9f12-a0e912011caa', 'quantity__1': 10, 'price__1': 100.0, 'timestamp__1': datetime.datetime(2024, 6, 22, 11, 42, 33), 'product_id__1': 'P001', 'customer_id__1': 'C0